In [ ]:
import pandas as pd
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import matplotlib.pyplot as plt

In [ ]:
# 1. 加载数据
data = pd.read_csv('Task2.csv')

# 数据结构
# iSub: 被试编号
# iTrial: 试次编号
# feature1, feature2, feature3, feature4: 四个特征
# choice: 被试的类别选择 (0或1)
# feedback: 反馈 (0: 错误, 1: 正确)

# 提取特征和其他变量
features = data[['feature1', 'feature2', 'feature3', 'feature4']].values
choices = data['choice'].values
feedback = data['feedback'].values

In [ ]:
# 2. 定义贝叶斯模型
with pm.Model() as model:
    # 信念参数: k 分割方法
    k = pm.DiscreteUniform('k', lower=1, upper=4)
    
    # 模糊性参数 beta
    beta = pm.HalfNormal('beta', sigma=1)
    
    # 定义刺激的中心点 (简化定义为固定的四个类别中心)
    # 假设类别C1在(0.25, 0.25, 0.25, 0.25), 类别C2在(0.75, 0.75, 0.75, 0.75)
    center_C1 = np.array([0.25, 0.25, 0.25, 0.25])
    center_C2 = np.array([0.75, 0.75, 0.75, 0.75])
    
    # 计算距离
    d_C1 = tt.sqrt(tt.sum(tt.square(features - center_C1), axis=1))
    d_C2 = tt.sqrt(tt.sum(tt.square(features - center_C2), axis=1))
    
    # 计算选择类别C1的概率 (软分类器)
    p_C1 = tt.nnet.softmax(tt.stack([-beta * d_C1, -beta * d_C2], axis=1))[:, 0]
    
    # 概率分布（类别选择）
    choice_likelihood = pm.Bernoulli('choice', p=p_C1, observed=choices)
    
    # 3. 似然函数：反馈与正确选择之间的一致性
    # 假设反馈是确定的，即选择正确类别会得到反馈1，选择错误类别会得到反馈0
    correct_choice = (choices == feedback).astype(int)
    feedback_likelihood = pm.Bernoulli('feedback', p=p_C1, observed=correct_choice)
    
    # 4. 采样后验分布
    trace = pm.sample(1000, tune=1000, cores=2, return_inferencedata=True)

In [ ]:
import pymc3 as pm

def fit_bayesian_model(df):
    # 获取条件
    conditions = df['condition'].unique()
    
    with pm.Model() as model:
        # 先验分布
        beta = pm.Uniform('beta', lower=0, upper=10)

        # 类别中心点
        centers = pm.Normal('centers', mu=0, sigma=1, shape=(4, 4))  # 4个类别，每个类别4个特征

        # 观察数据
        for i, row in df.iterrows():
            stimulus = row['stimulus']
            true_category = row['true_category']
            chosen_category = row['chosen_category']
            feedback = row['feedback']
            
            # 计算距离
            distances = pm.math.sqrt(pm.math.sum((centers - stimulus)**2, axis=1))
            probabilities = pm.math.exp(-beta * distances) / pm.math.sum(pm.math.exp(-beta * distances))

            # 选择类别的概率
            chosen_prob = probabilities[chosen_category]
            feedback_prob = pm.Bernoulli('feedback_{}'.format(i), p=chosen_prob, observed=feedback)

        # 后验采样
        trace = pm.sample(2000, tune=1000)

    return trace

In [4]:

import numpy as np
from scipy.special import softmax

class CategoryLearningModel:
    def __init__(self, K=4, beta=1.0):
        self.K = K  # Number of partition methods
        self.beta = beta  # Softness of partition
        self.theta = self.initialize_theta()
        
    def initialize_theta(self):
        # Initialize theta with uniform distribution over partitions and beta
        k = np.random.choice(self.K)
        beta = np.random.uniform(0.1, 10.0)  # Initialize beta between 0.1 and 10
        return {'k': k, 'beta': beta}
    
    def distance_to_centroid(self, x, centroid):
        return np.linalg.norm(x - centroid)
    
    def compute_probabilities(self, x):
        # Compute the probability of x belonging to each category
        k = self.theta['k']
        beta = self.theta['beta']
        
        # Define centroids for the 4 partition methods
        centroids = [
            np.array([0.5, 0.5, 0.5, 0.5]),
            np.array([0.5, 0.5, 0.5, 0.5]),
            np.array([0.5, 0.5, 0.5, 0.5]),
            np.array([0.5, 0.5, 0.5, 0.5])
        ]
        
        distances = [self.distance_to_centroid(x, centroids[i]) for i in range(4)]
        exp_neg_beta_distances = np.exp(-beta * np.array(distances))
        probabilities = exp_neg_beta_distances / np.sum(exp_neg_beta_distances)
        
        return probabilities
    
    def likelihood(self, x, c, r):
        probabilities = self.compute_probabilities(x)
        p_c_given_x = probabilities[c]
        
        if r == 1:
            p_r_given_c_x = 1.0
        else:
            p_r_given_c_x = 0.0
        
        return p_c_given_x * p_r_given_c_x
    
    def update_theta(self, x, c, r):
        # Example calculation for posterior
        posterior = np.random.rand(self.K)  # Replace with actual calculation
        posterior /= np.sum(posterior)  # Normalize to make it a probability distribution
        
        # Make sure posterior is an array with correct shape
        if posterior.ndim != 1 or len(posterior) != self.K:
            raise ValueError("Posterior must be a 1D array with length equal to K.")
        
        self.theta['k'] = np.random.choice(self.K, p=posterior)
        self.theta['beta'] = np.random.uniform(0.1, 10.0)
        
    def choose_category(self, x):
        probabilities = self.compute_probabilities(x)
        return np.argmax(probabilities)
    
    def fit(self, X, y, feedback):
        for x, c, r in zip(X, y, feedback):
            self.update_theta(x, c, r)
    
    def predict(self, X):
        return [self.choose_category(x) for x in X]

# Example usage
if __name__ == "__main__":
    model = CategoryLearningModel()
    
    # Generate some random data for demonstration
    X = np.random.rand(10, 4)  # 10 samples, 4 features each
    y = np.random.randint(0, 2, 10)  # Random categories (0 or 1)
    feedback = np.random.randint(0, 2, 10)  # Random feedback (0 or 1)
    
    model.fit(X, y, feedback)
    
    predictions = model.predict(X)
    print("Predictions:", predictions)

Predictions: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
